In [ ]:
from environment import environment
import random,sys,gc,warnings
from IPython.display import clear_output

import gymnasium 
from puzzle import easyBoard

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm
torch.autograd.set_detect_anomaly(True)
warnings.filterwarnings("ignore")

seed = 42
torch.manual_seed(seed)

In [ ]:
env = gymnasium.make("sudoku")
easyBoard_root = F.one_hot(easyBoard.clone().to(torch.int64),num_classes=10).to(torch.float32)
easyBoard_root = easyBoard_root.permute(-1,0,1).unsqueeze(0) # [1,10,9,9]
assert torch.equal(easyBoard,torch.argmax(easyBoard_root,dim=1).squeeze())

In [ ]:
representation = nn.Sequential( # - > latent state, torch.Size([1,32, 1, 1])
    nn.LazyConv2d(32,1,1,0),
    nn.ReLU(),
    nn.LazyConv2d(32,3,3,0),
    nn.ReLU(),
    nn.LazyConv2d(32,3,3,0),
    nn.ReLU()
)
representation.forward(easyBoard_root) # test init 
clear_output()

In [ ]:
class policy_value(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.prediction = nn.Sequential(
            nn.LazyConv2d(32,1,1,1),
            nn.ReLU(),
            nn.Flatten(start_dim=1), # torch.Size([1, 32])
            nn.LazyLinear(30),
            nn.ReLU() # torch.Size([1, 27])
        )
        self.value_head = nn.LazyLinear(1)
    
    def action_mask(self,logits: torch.Tensor):
        # altering softmax output so x and y in {0,8} and value in {1,9}
        assert logits.shape == torch.Size([3,10])
        mask = torch.zeros_like(logits,dtype=torch.bool)
        mask[0,-1] = True
        mask[1,-1] = True
        mask[-1,0] = True
        return logits.masked_fill(mask=mask,value=-float("inf"))
        
    def forward(self,x : torch.Tensor):
       x = self.prediction(x)
       p = self.action_mask(x.view(3,10))
       p = F.softmax(p,dim=-1) # policy out
       v = self.value_head(x)
       return p,v 

x = representation(easyBoard_root) # test init 
policy_value().forward(x)
clear_output()

In [70]:
import random
random.randint(0,9),random.randint(0,9),random.randint(1,9)

(6, 9, 8)

In [31]:
t = [3,4,6,7,7]
for n in t:
    d = list(map(lambda x : (x,),t))
d

[(3,), (4,), (6,), (7,), (7,)]

In [23]:
t : tuple = (1,)
type(t)

tuple

In [ ]:
d = {}
for n in s:
    d.update({n) : t})
print(d)

{3: [3, 4, 6, 7, 7], 4: [3, 4, 6, 7, 7], 6: [3, 4, 6, 7, 7], 7: [3, 4, 6, 7, 7]}


In [3]:
# takes a latent state and the high dimension action and output a reward 
dynamic = nn.Sequential( 
    nn.LazyConv2d(),
    nn.ReLU(),
    nn.LazyLinear(1)
)

NameError: name 'nn' is not defined

In [4]:
import numpy as np 

x = np.random.randint(1,19,(9,9)).sum()
x

839